# Pair Problem: Wookiee neighbors

Your rebel alliance team has been stranded on a remote planet in the Outer Rim. The memory banks of your ship have been wiped. You are the only surviving data scientist on the team.

Your location is near several planets that are largely inhabited by wookiees. Unfortunately, the different tribes of wookiees have different attitudes toward the alliance. It's important that your team know which tribe (represented by the color of the wookiee) will be on a planet or ship before exiting warp nearby. If you end up near a hostile wookiee tribe, you may not have time to reactive your warp drive before things turn sour.

The problem is that your databank is fried. Out of millions of ships and planets, you only know the location and color of a few hundred wookiee tribes.

Your team turns to you, the only one on the team that is capable of harnessing the power of The Force (data science). However, all of your tools were destroyed in the memory bank failure: no neural networks, no support vector machines, no models of any kind.

# Details

You must code, from scratch, a working KNN algorithm. Use the train-test split below to evaluate your model and then generate predictions for each of the observed wookiee ships in the holdout set.

- [train data](http://soph.info/metis/nyc18_ds15/wookiee-train.csv)
- [test data](http://soph.info/metis/nyc18_ds15/wookiee-test.csv)
- [holdout data](http://soph.info/metis/nyc18_ds15/wookiee-ho.csv)



In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

sns.set_style('whitegrid')

In [2]:
train_df = pd.read_csv('wookiee-train.csv', index_col=False)
test_df = pd.read_csv('wookiee-test.csv', index_col=False)
ho_df = pd.read_csv('wookiee-ho.csv', index_col=False)

In [3]:
train_df.drop('Unnamed: 0', axis=1, inplace=True)
test_df.drop('Unnamed: 0', axis=1, inplace=True)
ho_df.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
train_df

,wookieecolor,xcoord,ycoord,zcoord
0,red,-3.410692,0.854400,0.228154
1,red,0.350080,-0.751120,-1.845183
2,chartreuse,0.841712,-0.058204,0.246217
3,red,-0.646260,-1.821082,0.444616
4,blue,1.423538,2.269409,-1.061053
...,...,...,...,...
745,red,-0.116874,-3.500292,0.226594
746,white,0.916701,-0.152994,-0.133628
747,white,0.148534,-1.664906,-0.449369
748,white,0.336376,-0.507910,-0.389131


In [5]:
# store variables as numpy arrays

# Train Data
y_train = train_df['wookieecolor'].values
x_train = train_df[['xcoord', 'ycoord', 'zcoord']].values

# Test Data
y_test = test_df['wookieecolor'].values
x_test = test_df[['xcoord', 'ycoord', 'zcoord']].values

In [6]:
x_train

array([[-3.41069174,  0.85439963,  0.22815401],
       [ 0.35008039, -0.75112032, -1.84518281],
       [ 0.84171197, -0.05820395,  0.24621738],
       ...,
       [ 0.14853403, -1.66490633, -0.44936902],
       [ 0.33637617, -0.50791032, -0.38913138],
       [ 0.91512114, -1.65005094, -0.64136654]])

In [7]:
x_train.shape

(750, 3)

In [8]:
x_test.shape

(250, 3)

In [9]:
x_train.shape[0]

750

In [10]:
np.zeros((x_train.shape[0], x_test.shape[0])).shape

(750, 250)

In [11]:
train_df

,wookieecolor,xcoord,ycoord,zcoord
0,red,-3.410692,0.854400,0.228154
1,red,0.350080,-0.751120,-1.845183
2,chartreuse,0.841712,-0.058204,0.246217
3,red,-0.646260,-1.821082,0.444616
4,blue,1.423538,2.269409,-1.061053
...,...,...,...,...
745,red,-0.116874,-3.500292,0.226594
746,white,0.916701,-0.152994,-0.133628
747,white,0.148534,-1.664906,-0.449369
748,white,0.336376,-0.507910,-0.389131


In [12]:
for i, x1 in enumerate(x_train):
    print(i, x1)

0 [-3.41069174  0.85439963  0.22815401]
1 [ 0.35008039 -0.75112032 -1.84518281]
2 [ 0.84171197 -0.05820395  0.24621738]
3 [-0.64625967 -1.82108172  0.44461639]
4 [ 1.42353824  2.26940867 -1.06105339]
5 [-0.7108313  -3.65787424  0.02618126]
6 [ 0.90601217 -1.83265169 -0.96565912]
7 [-0.52976729 -0.60124404  0.5362572 ]
8 [ 1.1867509   2.18659425 -1.25052637]
9 [ 1.08338291 -0.15716913  0.57298385]
10 [-1.19708442 -0.64297923  0.22989119]
11 [ 1.41559271  0.44113602 -0.18091398]
12 [0.85797589 1.15153089 1.09757306]
13 [ 0.59387671 -0.48394804 -0.05777531]
14 [-0.59629516 -0.2256099   0.82317902]
15 [-1.28915065 -0.22303326 -0.94809673]
16 [-0.71185668 -0.63917107  0.70049822]
17 [-0.04548878  1.18717412 -0.01390351]
18 [-1.46716489 -0.26485992  0.59868405]
19 [-0.91211702 -1.01686688 -0.04886871]
20 [-1.34829285  1.43063947 -0.78721625]
21 [-2.32570554  0.50480665 -0.59345061]
22 [-1.11923667 -0.39257051  1.10607343]
23 [-1.36535648  1.83298116 -0.92749164]
24 [ 0.72156021  1.28550096 -

463 [-0.14592374 -0.58964777 -1.25857771]
464 [-0.37789753  0.95603183  1.11514685]
465 [-2.32757925  3.38413954 -1.42698849]
466 [ 1.74505524 -3.21156307 -1.03841492]
467 [-0.5895282  -0.020408   -1.87521281]
468 [-0.79922386 -2.09041894  1.0232509 ]
469 [-0.44868561 -0.6802836   0.76017435]
470 [ 1.53718717 -2.54944638  0.44644388]
471 [ 0.59520738 -1.13941801 -0.21997519]
472 [ 0.30702113 -1.11925824  0.28037508]
473 [ 2.74390288  1.89226583 -0.0294787 ]
474 [-1.52656669  0.03243928  1.81154854]
475 [-2.03910547  0.23931061  0.6128112 ]
476 [ 0.07510268 -0.34563116 -0.18575166]
477 [-1.49798922 -1.39404692  1.2480181 ]
478 [-0.41372295 -0.3189308   0.83312162]
479 [-0.97786738  0.21950024  0.84151884]
480 [-0.75773314  0.26546541  0.89534218]
481 [ 0.69011275 -0.72094228 -2.02185511]
482 [ 0.4121304  -0.87802879 -1.7256916 ]
483 [-1.13590127 -2.08633031  1.71128821]
484 [ 0.2101737  -1.27541655  0.79209988]
485 [ 1.91283691 -0.77186755 -1.92636622]
486 [-0.43612421  0.39923612  0.97

In [24]:
# define our distance metric. We’re using eucledian https://en.wikipedia.org/wiki/Euclidean_distance
def distance_own_sol(x1, x2):
    """
    Function takes a vector 
    """
    distance = 0.0
    for i in range(len(x1)-1):
        distance += (x1[i] - x2[i])**2
    return (distance)**0.5  # Applying a sqrt to return the eucledian distance (distance between 2 vectors)



def distance(x1, x2):
    
    # Euclidean Distance (Pythagoras Theorem for vectors)
    squared_diff = (x1 - x2)**2  # This works because of broadcasting
    sum_squared = sum(squared_diff)    # Summing up all the values (pythagoras theorem sums up all the squared differences)
    
    return (sum_squared)**0.5  # Applying a sqrt to return the eucledian distance (distance between 2 vectors)



# this is done
def pairwise_distances(X1, X2):
    
    dist = np.zeros( (X1.shape[0], X2.shape[0]))   # Returns a (X1.#_of_rows, X2.#_of_rows) zero matrix
    
    for i, x1 in enumerate(X1):  # For each row in X1 (x, y, z coordinates)
        
        for j, x2 in enumerate(X2):  # For every row of X1, compute distance of X1 with X2 
            
            dist[i, j] = distance(x1, x2)
        
    return dist   # returns a matrix of the distance between index i of X1 and index j of X2

In [25]:
pairwise_distances(x_train, x_test).shape

(750, 250)

In [44]:
def knn_predict(x_test_pt, x_train, y_train, k=2):
    
    # step 1: compute the pairwise distances :slightly_smiling_face:
    d_mat = pairwise_distances(x_test_pt, x_train)
    print('d_mat: ' d_mat)
    
    # step 2: use np.argsort to give us the indices of the smallest value in each row :slightly_smiling_face:
    d_mat_indces = np.argsort(d_mat)[:, :k]   #argsort gives you the indexes of the values itself
    print('d_mat_indices: ' d_mat_indces)
    
    # I want every single row and only up to k columns
    
    # step 3: picks out the y value for each index :slightly_smiling_face:
    votes = np.take(y_train, d_mat_indices, axis=None)
    print('votes:', votes)
    
    # step 4: do majority voting :slightly_smiling_face:
    y_pred = np.array([Counter(row).most_common(1)[0][0]]) for row in votes
    print('y_pred: ' y_pred)
    return y_pred

In [30]:
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier

# Scaling Data before input into algo
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)


#"""
# Before making any actual predictions, it is always a 
# good practice to scale the features so that all of them can be uniformly evaluated. 
# Wikipedia explains the reasoning pretty well:
# Since the range of values of raw data varies widely, in some machine learning algorithms, 
# objective functions will not work properly without normalization. 
# For example, the majority of classifiers calculate the distance between two points by the Euclidean distance. 
# If one of the features has a broad range of values, the distance will be governed by this particular feature. 
# Therefore, the range of all features should be normalized so that each feature contributes 
# approximately proportionately to the final distance.

# The gradient descent algorithm (which is used in neural network training and other machine 
# learning algorithms) also converges faster with normalized features.
#"""

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
# Scaling data before input into algo

knn = KNeighborsClassifier()
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)  # Create a prediction for an unknown set of x values
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        blue       0.74      0.64      0.69        50
  chartreuse       0.67      0.48      0.56        29
         red       0.82      0.84      0.83        97
       white       0.70      0.82      0.76        74

    accuracy                           0.75       250
   macro avg       0.73      0.70      0.71       250
weighted avg       0.75      0.75      0.75       250




# Possible extensions:

 * Does your solution work for any number of features in the training data sets?
 * Does your solution handle ties?
 * Can you add another parameter, `k`, to your solution so that it uses the `k` nearest Wookiees instead of only the nearest Wookiee?
 * Can you add to your solution so that it has reasonable behavior if `y_train` is numeric?

An extension of another kind:

 * Are you confident that your solution is correct? How can you ensure that it is, and check that it stays correct in the future?